In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from itertools import combinations_with_replacement

#### Data Loading

In [14]:
df=pd.read_csv("../zuu crew scores.csv")
df=df[df['CourseName']=='Foundations of ML']
del df['MemberName'] , df['CourseName']
df.head()

,EducationLevel,Attendance,TotalHours,AssignmentsCompleted,HackathonParticipation,GitHubScore,PeerReviewScore,CapstoneScore
0,3,79.9,43.7,2,0,62.8,5.0,45.3
1,2,76.8,95.6,6,0,87.4,2.7,78.8
2,3,96.6,75.9,8,0,98.4,2.8,65.4
4,2,83.2,24.0,6,0,41.8,4.2,40.1
7,3,86.5,88.0,5,0,23.9,1.3,68.2


In [15]:
Y = df['CapstoneScore'].values.reshape(-1, 1)
X_df = df.drop(columns=['CapstoneScore']).values

# Calculate mean and std for each feature (column)
mean = X_df.mean(axis=0)
std = X_df.std(axis=0)
X = (X_df - mean) / std

In [16]:
X.shape

(72, 7)

#### Polynominal Features

In [17]:
def polynomial_features(X, degree=2):
    n_samples, n_features = X.shape
    features = [np.ones((n_samples, 1))]  # bias term (x^0)
    
    for deg in range(1, degree + 1):
        for comb in combinations_with_replacement(range(n_features), deg):
            term = np.prod(X[:, comb], axis=1).reshape(-1, 1)
            features.append(term)
    
    return np.hstack(features)

In [18]:
#degree = 2  # change this to any degree you want
#X_poly = polynomial_features(X, degree)
#X_poly

#beta = np.linalg.pinv(X_poly.T @ X_poly) @ (X_poly.T @ Y)

#Y_pred = X_poly @ beta

#### Fit polynominal Feature

In [19]:

def fit_polynomial(X, y, degree=2, learning_rate=0.01, epochs=1000):
    # Expand features
    X_poly = polynomial_features(X, degree)
    
    n_samples, n_features = X_poly.shape
    beta = np.zeros((n_features, 1))  # initialize coefficients
    
    # Gradient descent loop
    for _ in range(epochs):
        y_pred = X_poly @ beta
        error = y_pred - y
        gradient = (2 / n_samples) * (X_poly.T @ error)
        beta -= learning_rate * gradient  # update rule
    
    return beta

#### Predict Function

In [20]:
# Predict function
def predict_polynomial(X, beta, degree):
    X_poly = polynomial_features(X, degree)
    return X_poly @ beta

In [21]:
# Fit model
beta = fit_polynomial(X, Y, degree=2, learning_rate=0.001, epochs=5000)

# Predict
y_pred = predict_polynomial(X, beta, degree=2)

#### Model Evaluation

In [22]:
Y_hat = predict_polynomial(X, beta, degree=2)

In [23]:
def MSE(Y,Y_hat):
    e_i = Y - Y_hat
    return np.mean(e_i**2)

def MAE(Y,Y_hat):
    e_i = Y - Y_hat
    return np.mean(np.abs(e_i))

def R2_Score(Y,Y_hat):
    y_mean = np.mean(Y)
    e_i = Y - Y_hat
    std_i = Y - y_mean

    E = np.sum(e_i**2) / np.sum(std_i ** 2)
    return 1 - E

def RMSE(Y,Y_hat):
    e_i = Y - Y_hat
    return np.sqrt(np.mean((Y - Y_hat) ** 2))

In [24]:
mse = MSE(Y, Y_hat)
mae = MAE(Y, Y_hat)
r2 = R2_Score(Y, Y_hat)
rmse = RMSE(Y,Y_hat)

print(f"MSE : {mse}")
print(f"MAE : {mae}")
print(f"R2 Score : {r2}")
print(f"RMSE : {rmse}")

MSE : 31.5042147461053
MAE : 4.207147836313957
R2 Score : 0.8897408570568401
RMSE : 5.612861547027977
